### Most feedback is not explicit but implicit in real-world scenarios.

- Non-observed user-item pairs which may be predictive for users' interests are totally ignored in matrix factorization and AutoRec. These models are not suitable for personalized ranking tasks.

- We'll discuss models targeting at generating ranked recommendation lists from implicit feedback.

- Personalized ranking models can be optimized with pointwise, pairwise or listwise approaches.

- Pointwise approaches considers a single interaction at a time and train a classifier or a regressor to predict individual preferences. Matrix factorization and AutoRec are optimized with pointwise objectives.

- Pairwise approaches consider a pair of items for each user and aim to approximate the optimal ordering for that pair.

- Listwise approaches approximate the ordering of the entire list of items.

#### In this section, we will introduce two pairwise objectives/losses

### Bayesian Personalized Ranking Loss and its Implementation

In formal, the training data is constructed by tuples in the form of $(u, i, j)$, which represents that the user $u$ prefers the item $i$ over the item $j$.

<img src="https://classic.d2l.ai/_images/rec-ranking.svg" />

$\begin{split}\begin{aligned}
\text{BPR-OPT} : &= \sum_{(u, i, j \in D)} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj}) - \lambda_\Theta \|\Theta \|^2
\end{aligned}\end{split}$

In [ ]:
!pip install mxnet-cu101==1.7.0

In [ ]:
!pip install d2l==0.17.5

In [3]:
from mxnet import gluon, np, npx

npx.set_np()

In [4]:
class BPRLoss(gluon.loss.Loss):
    def __init__(self, weight=None, batch_axis=0, **kwargs):
        super(BPRLoss, self).__init__(weight=None, batch_axis=0, **kwargs)

    def forward(self, positive, negative):
        distances = positive - negative
        loss = - np.sum(np.log(npx.sigmoid(distances)), 0, keepdims=True)
        return loss

### Hinge Loss and its Implementation

$\sum_{(u, i, j \in D)} \max( m - \hat{y}_{ui} + \hat{y}_{uj}, 0)$, where $m$ is the safety margin size.

In [5]:
class HingeLossbRec(gluon.loss.Loss):
    def __init__(self, weight=None, batch_axis=0, **kwargs):
        super(HingeLossbRec, self).__init__(weight=None, batch_axis=0,
                                            **kwargs)

    def forward(self, positive, negative, margin=1):
        distances = positive - negative
        loss = np.sum(np.maximum(- distances + margin, 0))
        return loss